## Team members:

- Mahmoud Ali
- Hossam Assayed Khairullah
- Youssef Wagih

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.metrics import r2_score, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
train_df = pd.read_csv('/kaggle/input/bike-sharing-demand/train.csv')
train_df.head()

In [ ]:
test_df = pd.read_csv('/kaggle/input/bike-sharing-demand/test.csv')
test_df.head()

In [ ]:
train_df.drop(['casual','registered'],1,inplace=True)

In [ ]:
train_df['datetime'] = pd.to_datetime(train_df['datetime'])
test_df['datetime'] = pd.to_datetime(test_df['datetime'])
train_df['Month']=pd.DatetimeIndex(train_df['datetime']).month
test_df['Month']=pd.DatetimeIndex(test_df['datetime']).month
train_df['Year']=pd.DatetimeIndex(train_df['datetime']).year
test_df['Year']=pd.DatetimeIndex(test_df['datetime']).year
train_df['WeekDay']=pd.DatetimeIndex(train_df['datetime']).weekday
test_df['WeekDay']=pd.DatetimeIndex(test_df['datetime']).weekday
train_df['Hour']=pd.DatetimeIndex(train_df['datetime']).hour
test_df['Hour']=pd.DatetimeIndex(test_df['datetime']).hour

In [ ]:
categorical_cols=['season','holiday','workingday','weather', 'Month','WeekDay','Hour']
numerical_cols=['temp','atemp','humidity','windspeed']
label='count'

## Custom Transfromer

In [ ]:
#removing atemp column as it is highly correlated with temp column
x=train_df.drop(['count','datetime','atemp'],1) 
xtest=test_df.drop(['datetime','atemp'],1)
y=train_df['count']
xt,xv,yt,yv=train_test_split(x,y,test_size=0.2,random_state=101)

In [ ]:
def RF_model(xt1,xv1,yt,yv,md=None,mss=2,n_est=100,al=0):
    
    model=RandomForestRegressor(max_depth=md, random_state=0,min_samples_split=mss,n_estimators=n_est,ccp_alpha=al)
    model.fit(xt1,yt)
    
    ypt = model.predict(xt1)
    ypv = model.predict(xv1)
    
    print('training r2:',r2_score(yt,ypt))
    print('Validation r2:',r2_score(yv,ypv))
    print('training rmsle:',np.sqrt(mean_squared_log_error(yt,ypt)))
    print('validation rmsle:',np.sqrt(mean_squared_log_error(yv,ypv)))
    return model

In [ ]:
RF_model(xt,xv,yt,yv)

In [ ]:
model = RF_model(xt,xv,yt,yv)

In [ ]:
y_predict = model.predict(xtest)

In [ ]:
sub = pd.read_csv('/kaggle/input/bike-sharing-demand/sampleSubmission.csv')
sub.head()

In [ ]:
sub['count'] = y_predict
sub.head()

In [ ]:
sub.to_csv('/kaggle/working/submission.csv', index=False)